In [20]:
from datetime import datetime
from typing import List, Dict, Any, Union
import time


class Account:
    """Базовый класс для банковского счета."""
    
    def __init__(self, account_holder: str, balance: float = 0):
        """
        Инициализация банковского счета.
        
        Аргументы:
            account_holder (str): Имя владельца счета
            balance (float, optional): Начальный баланс. По умлочанию = 0.
            
        Ошибка:
            ValueError: Если баланс отрицательный
        """
        self.holder = account_holder
        
        if balance < 0:
            raise ValueError("Баланс не может быть отрицательным!")
        
        self._balance = balance
        self.operations_history: List[Dict[str, Any]] = []
        
        # Устанавливаем начальный баланс
        if balance > 0:
            self._add_operation(
                operation_type="initial_deposit",
                amount=self._balance,
                status="success",
                message=f"Начальный баланс: {self._balance} руб."
            )
    
    def _add_operation(self, operation_type: str, amount: float, 
                      status: str, message: str = "") -> None:
        """
        Добавляет запись об операции в историю.
        
        Аргументы:
            operation_type (str): Тип операции
            amount (float): Сумма операции
            status (str): Статус операции ('success' или 'fail')
            message (str, optional): Дополнительное сообщение. По умолчанию пусто "".
        """
        operation = {
            "type": operation_type,
            "amount": amount,
            "datetime": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "balance_after": self._balance,
            "status": status,
            "message": message
        }
        self.operations_history.append(operation)
    
    def deposit(self, amount: float) -> bool:
        """
        Пополнение счета.
        
        Аргументы:
            amount (float): Сумма для пополнения
            
        Возвращает:
            True если операция успешна, False в противном случае
        """
        if amount <= 0:
            self._add_operation(
                operation_type="deposit",
                amount=amount,
                status="fail",
                message=f"Нельзя пополнить на {amount} руб. Сумма должна быть положительной."
            )
            return False
        
        self._balance += amount
        self._add_operation(
            operation_type="deposit",
            amount=amount,
            status="success",
            message=f"Баланс пополнен на {amount} руб."
        )
        return True
    
    def withdraw(self, amount: float) -> bool:
        """
        Снятие средств со счета.
        
        Аргументы:
            amount (float): Сумма для снятия
            
        Возвращает:
            True если операция успешна, False в противном случае
        """
        if amount <= 0:
            self._add_operation(
                operation_type="withdraw",
                amount=amount,
                status="fail",
                message=f"Нельзя снять {amount} руб. Сумма должна быть положительной."
            )
            return False
        
        if amount > self._balance:
            self._add_operation(
                operation_type="withdraw",
                amount=amount,
                status="fail",
                message=f"Недостаточно средств. Попытка снять {amount} руб. при балансе {self._balance} руб."
            )
            return False
        
        self._balance -= amount
        self._add_operation(
            operation_type="withdraw",
            amount=amount,
            status="success",
            message=f"Снято {amount} руб."
        )
        return True
    
    def get_balance(self) -> float:
        """
        Возвращает текущий баланс.
        
        """
        return self._balance
    
    def get_history(self, formatted: bool = False) -> Union[List[Dict[str, Any]], str]:
        """
        Возвращает историю операций.
        
        Аргументы:
            formatted (bool, optional): Форматировать вывод. Defaults to False.
            
        Возвращает:
            Историю операций
        """
        if not formatted:
            return self.operations_history
        
        # Форматированный вывод
        result = []
        result.append(f"\nИстория операций счета: {self.holder}")
        result.append("=" * 60)
        
        for i, op in enumerate(self.operations_history, 1):
            result.append(
                f"{i}. [{op['datetime']}] {op['type'].upper():15} "
                f"Сумма: {op['amount']:8.2f} руб. | "
                f"Статус: {op['status']:7} | "
                f"Баланс: {op['balance_after']:8.2f} руб. | "
                f"{op['message']}"
            )
        
        result.append("=" * 60)
        result.append(f"Текущий баланс: {self._balance:.2f} руб.")
        return "\n".join(result)
    
    def __str__(self) -> str:
        """Строковое представление счета."""
        return f"Счет: {self.holder}, Баланс: {self._balance:.2f} руб."

acc_ = Account("Дмитрий", 5000.0)

acc.deposit(1000.0)
time.sleep(1)
acc.withdraw(2000.0)
acc.withdraw(5000.0)  # неуспешно
acc.deposit(3000.0)
acc.withdraw(1000.0)

# Выводим историю операций
print(acc.get_history(formatted=True))

print(f"\nИтоговый баланс: {acc.get_balance()} руб.")
   


История операций счета: Дмитрий
1. [2025-12-07 13:36:40] INITIAL_DEPOSIT Сумма:  5000.00 руб. | Статус: success | Баланс:  5000.00 руб. | Начальный баланс: 5000.0 руб.
2. [2025-12-07 13:36:40] DEPOSIT         Сумма:  1000.00 руб. | Статус: success | Баланс:  6000.00 руб. | Баланс пополнен на 1000.0 руб.
3. [2025-12-07 13:36:41] WITHDRAW        Сумма:  2000.00 руб. | Статус: success | Баланс:  4000.00 руб. | Снято 2000.0 руб.
4. [2025-12-07 13:36:41] WITHDRAW        Сумма:  5000.00 руб. | Статус: fail    | Баланс:  4000.00 руб. | Недостаточно средств. Попытка снять 5000.0 руб. при балансе 4000.0 руб.
5. [2025-12-07 13:36:41] DEPOSIT         Сумма:  3000.00 руб. | Статус: success | Баланс:  7000.00 руб. | Баланс пополнен на 3000.0 руб.
6. [2025-12-07 13:36:41] WITHDRAW        Сумма:  1000.00 руб. | Статус: success | Баланс:  6000.00 руб. | Снято 1000.0 руб.
Текущий баланс: 6000.00 руб.

Итоговый баланс: 6000.0 руб.


In [28]:

class CreditAccount(Account):
    """Кредитный счет с поддержкой кредитного лимита."""
    
    def __init__(self, account_holder: str, balance: float = 0, 
                 credit_limit: float = 0):
        """
        Инициализация кредитного счета.
        
        Аргументы:
            account_holder (str): Имя владельца счета
            balance (float, optional): Начальный баланс. Defaults to 0.
            credit_limit (float, optional): Кредитный лимит. Defaults to 0.
            
        Возвращает:
            Ошибку если баланс меньше отрицательного кредитного лимита
        """
        if balance < -credit_limit:
            raise ValueError(
                f"Баланс ({balance} руб.) не может быть меньше кредитного лимита "
                f"(-{credit_limit} руб.)"
            )
        
        super().__init__(account_holder, balance)
        self._credit_limit = credit_limit
        
        # Обновляем первую запись в истории операций для кредитного счета
        if self.operations_history:
            self.operations_history[0]["message"] = (
                f"Начальный баланс: {balance} руб., "
                f"Кредитный лимит: {credit_limit} руб."
            )
    
    def get_available_credit(self) -> float:
        """
        Возвращает доступный кредит.
        
        Возвращает:
            Сумма доступного кредита
        """
        return self._balance + self._credit_limit
    
    def withdraw(self, amount: float) -> bool:
        """
        Снятие средств с кредитного счета.
        
        Аргументы:
            amount (float): Сумма для снятия
            
        Возвращает:
            True если операция успешна, False в противном случае
        """
        if amount <= 0:
            self._add_operation(
                operation_type="withdraw",
                amount=amount,
                status="fail",
                message=f"Нельзя снять {amount} руб. Сумма должна быть положительной."
            )
            return False
        
        if amount > self.get_available_credit():
            self._add_operation(
                operation_type="withdraw",
                amount=amount,
                status="fail",
                message=(
                    f"Превышен лимит. Доступно: {self.get_available_credit():.2f} руб., "
                    f"попытка снять: {amount} руб."
                )
            )
            return False
        
        # Проверяем, используются ли кредитные средства
        uses_credit = amount > self._balance
        was_balance = self._balance
        
        self._balance -= amount
        
        if uses_credit:
            credit_used = amount - was_balance
            self._add_operation(
                operation_type="withdraw",
                amount=amount,
                status="success",
                message=(
                    f"Снято {amount} руб. (использовано кредитных средств: "
                    f"{credit_used:.2f} руб.)"
                )
            )
        else:
            self._add_operation(
                operation_type="withdraw",
                amount=amount,
                status="success",
                message=f"Снято {amount} руб."
            )
        
        return True
    
    def get_balance_info(self) -> str:
        """
        Возвращает полную информацию о балансе.
        
        Возвращает:
            Информация о балансе и кредитном лимите
        """
        return (
            f"Баланс: {self._balance:.2f} руб. | "
            f"Кредитный лимит: {self._credit_limit:.2f} руб. | "
            f"Доступно средств: {self.get_available_credit():.2f} руб."
        )
    
    def __str__(self) -> str:
        """Строковое представление кредитного счета."""
        return (
            f"Кредитный счет: {self.holder}, "
            f"Баланс: {self._balance:.2f} руб., "
            f"Кредитный лимит: {self._credit_limit:.2f} руб., "
            f"Доступно: {self.get_available_credit():.2f} руб."
        )

print("=" * 60)
print("ДЕМОНСТРАЦИЯ РАБОТЫ КЛАССОВ")
print("=" * 60)
    
 # Тестирование базового счета
print("\n1. Тестирование базового счета:")
print("-" * 40)
    
try:
    # Пытаемся создать счет с отрицательным балансом (должна быть ошибка)
    acc_negative = Account("Иван", -100)
except ValueError as e:
    print(f"Ошибка при создании счета: {e}")
    
# Создаем корректный счет
acc = Account("Дмитрий", 5000.0)
print(f"Создан счет: {acc}")
    
# Выполняем операции
acc.deposit(1000.0)
time.sleep(1)
acc.withdraw(2000.0)
time.sleep(1)
acc.withdraw(5000.0)  # неуспешно (недостаточно средств)
time.sleep(1)
acc.deposit(3000.0)
time.sleep(1)
acc.withdraw(-500.0)  # неуспешно (отрицательная сумма)
time.sleep(1)
acc.withdraw(1000.0)
    
# Выводим историю операций
print(acc.get_history(formatted=True))
    
# Тестирование кредитного счета
print("\n\n2. Тестирование кредитного счета:")
print("-" * 40)
    
# Создаем кредитный счет
credit_acc = CreditAccount("Анна", 1000.0, 5000.0)
print(f"Создан кредитный счет: {credit_acc}")
print(f"Информация о балансе: {credit_acc.get_balance_info()}")
    
# Выполняем операции
credit_acc.deposit(2000.0)
time.sleep(1)
credit_acc.withdraw(5000.0)  # успешно (используется кредит)
time.sleep(1)
credit_acc.withdraw(3000.0)  # неуспешно (превышен лимит)
time.sleep(1)
credit_acc.deposit(1000.0)
time.sleep(1)
credit_acc.withdraw(1500.0)  # успешно
    
# Выводим историю операций
print(credit_acc.get_history(formatted=True))
    
print("\n" + "=" * 60)
print("ТЕСТИРОВАНИЕ ЗАВЕРШЕНО")
print("=" * 60)


    

ДЕМОНСТРАЦИЯ РАБОТЫ КЛАССОВ

1. Тестирование базового счета:
----------------------------------------
Ошибка при создании счета: Баланс не может быть отрицательным!
Создан счет: Счет: Дмитрий, Баланс: 5000.00 руб.

История операций счета: Дмитрий
1. [2025-12-07 13:51:00] INITIAL_DEPOSIT Сумма:  5000.00 руб. | Статус: success | Баланс:  5000.00 руб. | Начальный баланс: 5000.0 руб.
2. [2025-12-07 13:51:00] DEPOSIT         Сумма:  1000.00 руб. | Статус: success | Баланс:  6000.00 руб. | Баланс пополнен на 1000.0 руб.
3. [2025-12-07 13:51:01] WITHDRAW        Сумма:  2000.00 руб. | Статус: success | Баланс:  4000.00 руб. | Снято 2000.0 руб.
4. [2025-12-07 13:51:02] WITHDRAW        Сумма:  5000.00 руб. | Статус: fail    | Баланс:  4000.00 руб. | Недостаточно средств. Попытка снять 5000.0 руб. при балансе 4000.0 руб.
5. [2025-12-07 13:51:03] DEPOSIT         Сумма:  3000.00 руб. | Статус: success | Баланс:  7000.00 руб. | Баланс пополнен на 3000.0 руб.
6. [2025-12-07 13:51:04] WITHDRAW        